In [306]:
import torch
from torch import nn
from d2l import torch as d2l

In [307]:
def corr2d(X, K):
    # 计算二维互相关计算
    h, w = K.shape
    print(h, w)
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [308]:
x = torch.tensor([
    [0.0, 1.0, 2.0],
    [3.0, 4.0, 5.0],
    [6.0, 7.0, 8.0],
])
y = torch.tensor([
    [0.0, 1.0],
    [2.0, 3.0]
])
print(corr2d(x, y))


2 2
tensor([[19., 25.],
        [37., 43.]])


In [309]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        print(self.weight)
        return corr2d(x, self.weight) + self.bias

In [310]:
conv2d = Conv2d([2, 2])
conv2d(x)

Parameter containing:
tensor([[0.8619, 0.6813],
        [0.4223, 0.9764]], requires_grad=True)
2 2


tensor([[ 5.8540,  8.7959],
        [14.6798, 17.6218]], grad_fn=<AddBackward0>)

In [311]:
X = torch.ones(8, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [312]:
K_col = torch.tensor([
    [1.0, -1.0]
])
K_row = torch.tensor([
    [1.0],
    [-1.0]
])
K_col, K_row

(tensor([[ 1., -1.]]),
 tensor([[ 1.],
         [-1.]]))

In [313]:
Y = corr2d(X, K_col)
corr2d(X, K_col), corr2d(X, K_row)

1 2
1 2
2 1


(tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]))

In [314]:
conv2d = nn.Conv2d(
    1,
    1,
    kernel_size=(1, 2),
    bias=False
)

lr = 0.01
X = X.reshape(1, 1, 8, 8)
Y = Y.reshape(1, 1, 8, 7)

for i in range(1000):
    Y_hat = conv2d(X)
    loss = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data -= lr * conv2d.weight.grad
    print(f'batch {i + 1}:, loss: {loss.sum():.10f}')

batch 1:, loss: 35.2514762878
batch 2:, loss: 21.0448722839
batch 3:, loss: 14.6961193085
batch 4:, loss: 10.3634548187
batch 5:, loss: 7.3122091293
batch 6:, loss: 5.1594848633
batch 7:, loss: 3.6405320168
batch 8:, loss: 2.5687596798
batch 9:, loss: 1.8125169277
batch 10:, loss: 1.2789115906
batch 11:, loss: 0.9024001360
batch 12:, loss: 0.6367335320
batch 13:, loss: 0.4492790997
batch 14:, loss: 0.3170113862
batch 15:, loss: 0.2236832529
batch 16:, loss: 0.1578309983
batch 17:, loss: 0.1113655120
batch 18:, loss: 0.0785795376
batch 19:, loss: 0.0554456972
batch 20:, loss: 0.0391224511
batch 21:, loss: 0.0276048221
batch 22:, loss: 0.0194779877
batch 23:, loss: 0.0137436399
batch 24:, loss: 0.0096975090
batch 25:, loss: 0.0068425653
batch 26:, loss: 0.0048281252
batch 27:, loss: 0.0034067365
batch 28:, loss: 0.0024037836
batch 29:, loss: 0.0016961175
batch 30:, loss: 0.0011967820
batch 31:, loss: 0.0008444493
batch 32:, loss: 0.0005958485
batch 33:, loss: 0.0004204335
batch 34:, loss

In [316]:
conv2d.weight.data.reshape(1, 2)

tensor([[ 1.0000, -1.0000]])